**Task: Create FPR/FNR/Accuracy graphs for different mixing rates for the Folktable**


Custom Prediction Task: ACSIncomeNew 

In [1]:
import numpy as np
import pandas as pd

import folktables
ACSIncomeNew = folktables.BasicProblem(
    features=[
        'AGEP',
        'COW',
        'SCHL',
        'MAR',
        'OCCP',
        'POBP',
        'RELP',
        'WKHP',
        'SEX',
        'RAC1P',
    ],
    target='PINCP',
    target_transform=lambda x: x > 50000,    
    group='SEX',
    preprocess=folktables.adult_filter,
    postprocess=lambda x: np.nan_to_num(x, -1),
)

In [59]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from tqdm import tqdm

from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler
# from sklearn.metrics import plot_roc_curve, roc_curve, roc_auc_score,auc
from sklearn.metrics import RocCurveDisplay, roc_curve, roc_auc_score,auc


import folktables

from folktables import ACSDataSource, ACSEmployment

data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_ca = data_source.get_data(states=["CA"], download=True)


MemoryError: Unable to allocate 2.89 MiB for an array with shape (378817,) and data type float64

In [3]:
pred_label = 'PINCP'
pincp = acs_data[[pred_label]]


In [4]:
pincp

,PINCP
0,-1500.0
1,1600.0
2,10000.0
3,0.0
4,0.0
...,...
28112,18700.0
28113,0.0
28114,11100.0
28115,7200.0


In [5]:
acs_ca.iloc[0]

RT                      P
SERIALNO    2018GQ0000049
DIVISION                6
SPORDER                 1
PUMA                 1600
                ...      
PWGTP76                75
PWGTP77                80
PWGTP78                74
PWGTP79                 7
PWGTP80                72
Name: 0, Length: 286, dtype: object

In [6]:
acs_data.shape

(3236107, 286)

In [7]:
features, label, group = ACSIncomeNew.df_to_numpy(acs_data)


In [8]:
# X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
#     features, label, group, test_size=0.2, random_state=0)

In [9]:
x_labels = ACSIncomeNew.features

In [10]:
x_labels

['AGEP', 'COW', 'SCHL', 'MAR', 'OCCP', 'POBP', 'RELP', 'WKHP', 'SEX', 'RAC1P']

In [11]:
# df_new = balance_data(acs_data, ['f', 'm'], random_state=seed)


In [12]:
X = acs_data[x_labels]

In [13]:
X.shape

(3236107, 10)

In [14]:
X

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P
0,19,1.0,18.0,5,4720.0,13,17,NaN,2,1
1,18,1.0,18.0,5,4720.0,13,17,21.0,2,2
2,53,5.0,17.0,5,3605.0,18,16,40.0,1,1
3,28,NaN,19.0,5,NaN,40,16,NaN,1,1
4,25,1.0,12.0,5,9640.0,1,16,NaN,2,1
...,...,...,...,...,...,...,...,...,...,...
28112,47,1.0,16.0,1,8990.0,72,1,40.0,1,8
28113,18,NaN,19.0,5,NaN,72,2,NaN,2,8
28114,57,NaN,20.0,1,NaN,72,0,NaN,2,1
28115,66,NaN,12.0,1,NaN,72,1,NaN,1,1


In [15]:
y = acs_data[pred_label]

In [16]:
y

0        -1500.0
1         1600.0
2        10000.0
3            0.0
4            0.0
          ...   
28112    18700.0
28113        0.0
28114    11100.0
28115     7200.0
28116     7900.0
Name: PINCP, Length: 3236107, dtype: float64

In [17]:
y.shape

(3236107,)

In [18]:



def select_rows(df, cond='all', drop_sens=False):
    if cond == 'f':
        # Female
        df_sel = df[df['SEX']==2]
    elif cond == 'm':
        # Male
        df_sel = df[df['SEX']==1]
    elif cond == 'all':
        df_sel = df.copy()
    if drop_sens:
        if(cond == 'f') or (cond == 'm'):
            df_sel = df_sel.drop(columns=['SEX'])
            
    return df_sel

In [19]:
acs_data[acs_data['SEX']==1] # Male

,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,PWGTP,AGEP,...,PWGTP71,PWGTP72,PWGTP73,PWGTP74,PWGTP75,PWGTP76,PWGTP77,PWGTP78,PWGTP79,PWGTP80
2,P,2018GQ0000219,6,1,2000,3,1,1013097,118,53,...,117,121,123,205,208,218,120,19,123,18
3,P,2018GQ0000246,6,1,2400,3,1,1013097,43,28,...,43,76,79,77,80,44,46,82,81,8
7,P,2018GQ0000943,6,1,600,3,1,1013097,85,38,...,82,13,12,13,83,86,88,86,83,13
8,P,2018GQ0001008,6,1,2200,3,1,1013097,16,41,...,29,14,30,2,16,32,33,16,16,16
10,P,2018GQ0001151,6,1,2200,3,1,1013097,92,65,...,101,94,8,88,170,90,171,174,6,101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28107,P,2018HU1400280,0,1,502,9,72,1013097,162,39,...,216,52,136,147,50,176,166,146,52,148
28108,P,2018HU1400424,0,1,403,9,72,1013097,84,38,...,126,80,129,81,25,79,24,27,155,111
28112,P,2018HU1400717,0,2,201,9,72,1013097,205,47,...,290,378,173,192,324,83,239,437,129,283
28115,P,2018HU1400772,0,2,902,9,72,1013097,64,66,...,112,72,69,63,54,18,123,63,56,103


In [20]:
acs_data[acs_data['SEX']==2] # Female

,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,PWGTP,AGEP,...,PWGTP71,PWGTP72,PWGTP73,PWGTP74,PWGTP75,PWGTP76,PWGTP77,PWGTP78,PWGTP79,PWGTP80
0,P,2018GQ0000049,6,1,1600,3,1,1013097,75,19,...,140,74,73,7,76,75,80,74,7,72
1,P,2018GQ0000058,6,1,1900,3,1,1013097,75,18,...,76,78,7,76,80,78,7,147,150,75
4,P,2018GQ0000251,6,1,2701,3,1,1013097,16,25,...,4,2,29,17,15,28,17,30,15,1
5,P,2018GQ0000390,6,1,2400,3,1,1013097,25,30,...,28,28,1,26,47,25,3,3,26,27
6,P,2018GQ0000510,6,1,400,3,1,1013097,18,66,...,17,2,18,16,2,18,1,34,36,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28109,P,2018HU1400424,0,2,403,9,72,1013097,74,37,...,101,65,110,66,21,74,24,26,127,89
28110,P,2018HU1400599,0,1,804,9,72,1013097,84,76,...,79,154,87,133,136,137,141,89,90,85
28111,P,2018HU1400717,0,1,201,9,72,1013097,98,47,...,161,214,89,88,140,31,90,220,99,123
28113,P,2018HU1400717,0,3,201,9,72,1013097,228,18,...,352,317,202,182,283,75,191,448,191,282


In [21]:
acs_data[acs_data['SEX']==1].shape

(1584636, 286)

In [22]:
acs_data[acs_data['SEX']==2].shape

(1651471, 286)

In [23]:
def select_rows_p(df, select, p, random_state=42):
    df_m_sub = select_rows(df, 'm')
    df_f_sub = select_rows(df, 'f')
    m_sample_size, f_sample_size = int(len(df_m_sub)*(1-p)), int(len(df_m_sub)*p)
    np.random.seed(random_state)
    m_sample_idx = np.random.permutation(np.arange(len(df_m_sub)))[:m_sample_size]
    np.random.seed(random_state)
    f_sample_idx = np.random.permutation(np.arange(len(df_f_sub)))[:f_sample_size]
    df_m_sub = df_m_sub.iloc[m_sample_idx]
    df_f_sub = df_f_sub.iloc[f_sample_idx]
    return pd.concat([df_m_sub, df_f_sub])





In [24]:

    def process_missing_values(df, y_label, col_thresh=100, impute_method='drop'):
        """
        Description:
            Processes rows that don't have enough values, either by imputing or dropping
        
        Parameters:
            df (pd dataframe): The pandas dataframe.
            y_label (string): The pred_label.
            col_thresh (int): Requires that many non-NA values. Default threshold is 100.
            impute_method (string): Tells function which method of imputation to use to process missing values
        
        Returns:
            Dataframe with missing values gone, either imputed or dropped
        
        """
        
#         df_new = df.dropna(subset=y_label)
        df_new = df.dropna(axis=1, thresh=col_thresh)

        if impute_method == 'drop':
            df_new = df_new.dropna()  # Dropping all the rows with "any" missing value
        elif impute_method == 'mean':
            df_new = df_new.fillna(df_new.mean())
        elif impute_method == 'median':
            df_new = df_new.fillna(df_new.median())
        elif impute_method == 'knn':      
            df_num = df_new.select_dtypes(np.number)
            df_text = df_new.select_dtypes(exclude=[np.number])
            imputer = KNNImputer(n_neighbors=5)
            df_num = pd.DataFrame(imputer.fit_transform(df_num), columns=df_num.columns, index=df_num.index)
            df_new = pd.concat([df_text, df_num], axis=1)
        elif impute_method == 'iterative':
            df_num = df_new.select_dtypes(np.number)
            df_text = df_new.select_dtypes(exclude=[np.number])
            imputer = IterativeImputer()
            df_num = pd.DataFrame(imputer.fit_transform(df_num), columns=df_num.columns, index=df_num.index)
            df_new = pd.concat([df_text, df_num], axis=1)

        return df_new
    

In [32]:
def process_missing_values_partitioned(df, y_label, col_thresh=100, impute_method='drop', partition_size=1000):
    """
    Description:
        Processes rows that don't have enough values, either by imputing or dropping, in partitions
        
    Parameters:
        df (pd dataframe): The pandas dataframe.
        y_label (string): The pred_label.
        col_thresh (int): Requires that many non-NA values. Default threshold is 100.
        impute_method (string): Tells function which method of imputation to use to process missing values
        chunk_size (int): The size of each chunk for processing
        
    Returns:
        Dataframe with missing values gone, either imputed or dropped
        
    """
    partitions = [df[i:i + partition_size] for i in range(0, df.shape[0], partition_size)]
    processed_partitions = []

    for partition in partitions:
        df_partition = partition.dropna(axis=1, thresh=col_thresh)

        if impute_method == 'drop':
            df_partition = df_partition.dropna()  # Dropping all the rows with "any" missing value
        elif impute_method == 'mean':
            df_partition = df_partition.fillna(df_partition.mean())
        elif impute_method == 'median':
            df_partition = df_partition.fillna(df_partition.median())
        elif impute_method == 'knn':
            df_num = df_partition.select_dtypes(np.number)
            df_text = df_partition.select_dtypes(exclude=[np.number])
            imputer = KNNImputer(n_neighbors=5)
            df_num = pd.DataFrame(imputer.fit_transform(df_num), columns=df_num.columns, index=df_num.index)
            df_partition = pd.concat([df_text, df_num], axis=1)
        elif impute_method == 'iterative':
            df_num = df_chunk.select_dtypes(np.number)
            df_text = df_chunk.select_dtypes(exclude=[np.number])
            imputer = IterativeImputer()
            df_num = pd.DataFrame(imputer.fit_transform(df_num), columns=df_num.columns, index=df_num.index)
            df_partition = pd.concat([df_text, df_num], axis=1)

        processed_partitions.append(df_partition)

    return pd.concat(processed_partitions)

In [57]:
partitioned_df_new = process_missing_values(acs_data, 'PINCP', impute_method='mean')


MemoryError: Unable to allocate 2.05 GiB for an array with shape (85, 3236107) and data type float64

In [27]:
partitioned_df_new

,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,PWGTP,AGEP,...,MLPI,MLPJ,MLPK,VPS,ENG,LANP,FHINS5C,YOEP,DECADE,CITWP
28010,P,2018HU1396389,0,3,1101,9,72,1013097,265,26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28011,P,2018HU1396389,0,4,1101,9,72,1013097,98,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28012,P,2018HU1396389,0,5,1101,9,72,1013097,114,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28013,P,2018HU1396530,0,1,1102,9,72,1013097,123,39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28014,P,2018HU1396530,0,2,1102,9,72,1013097,186,19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28112,P,2018HU1400717,0,2,201,9,72,1013097,205,47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28113,P,2018HU1400717,0,3,201,9,72,1013097,228,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28114,P,2018HU1400772,0,1,902,9,72,1013097,46,57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28115,P,2018HU1400772,0,2,902,9,72,1013097,64,66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
def train_model(X_train, y_train, model='randomforest'):
    if model == 'randomforest':
        pred_model =  RandomForestClassifier(max_depth=5, min_samples_leaf=3, random_state=42)
    elif model == 'logistic':
        pred_model = LogisticRegression(multi_class='ovr', C = 1)
    elif model == 'svm': 
        pred_model = svm.SVC(kernel='rbf') 

    pred_model.fit(X_train, y_train)

    return pred_model 


In [29]:
def balance_data(df, groups, random_state=42):
    
    df0 = select_rows(df, groups[0])
    df1 = select_rows(df, groups[1])
    size0, size1 = len(df0), len(df1)
    if size0 > size1:
        df_tmp = df0
        df0 = df1
        df1 = df_tmp
    size0, size1 = len(df0), len(df1)
    np.random.seed(random_state)
    sample_idx = np.random.permutation(np.arange(size1))[:size0]
    df_new = pd.concat([df0,  df1.iloc[sample_idx]])

    return df_new

In [30]:
acs_data.shape

(3236107, 286)

In [38]:
partitioned_df_new.shape

(105, 280)

In [39]:
partitioned_df_new[partitioned_df_new['SEX']==2].shape

(59, 280)

In [40]:
partitioned_df_new[partitioned_df_new['SEX']==2]

,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,PWGTP,AGEP,...,MLPI,MLPJ,MLPK,VPS,ENG,LANP,FHINS5C,YOEP,DECADE,CITWP
28010,P,2018HU1396389,0,3,1101,9,72,1013097,265,26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28013,P,2018HU1396530,0,1,1102,9,72,1013097,123,39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28016,P,2018HU1396899,0,1,804,9,72,1013097,79,75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28019,P,2018HU1397010,0,1,402,9,72,1013097,61,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28021,P,2018HU1397078,0,1,101,9,72,1013097,89,46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28025,P,2018HU1397098,0,1,901,9,72,1013097,146,66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28026,P,2018HU1397152,0,1,806,9,72,1013097,25,44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28028,P,2018HU1397152,0,3,806,9,72,1013097,30,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28029,P,2018HU1397185,0,1,202,9,72,1013097,46,71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28031,P,2018HU1397252,0,1,403,9,72,1013097,107,68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
acs_data[acs_data['SEX']==2]

,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,PWGTP,AGEP,...,PWGTP71,PWGTP72,PWGTP73,PWGTP74,PWGTP75,PWGTP76,PWGTP77,PWGTP78,PWGTP79,PWGTP80
0,P,2018GQ0000049,6,1,1600,3,1,1013097,75,19,...,140,74,73,7,76,75,80,74,7,72
1,P,2018GQ0000058,6,1,1900,3,1,1013097,75,18,...,76,78,7,76,80,78,7,147,150,75
4,P,2018GQ0000251,6,1,2701,3,1,1013097,16,25,...,4,2,29,17,15,28,17,30,15,1
5,P,2018GQ0000390,6,1,2400,3,1,1013097,25,30,...,28,28,1,26,47,25,3,3,26,27
6,P,2018GQ0000510,6,1,400,3,1,1013097,18,66,...,17,2,18,16,2,18,1,34,36,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28109,P,2018HU1400424,0,2,403,9,72,1013097,74,37,...,101,65,110,66,21,74,24,26,127,89
28110,P,2018HU1400599,0,1,804,9,72,1013097,84,76,...,79,154,87,133,136,137,141,89,90,85
28111,P,2018HU1400717,0,1,201,9,72,1013097,98,47,...,161,214,89,88,140,31,90,220,99,123
28113,P,2018HU1400717,0,3,201,9,72,1013097,228,18,...,352,317,202,182,283,75,191,448,191,282


In [58]:
seed =777
balanced = balance_data(acs_data, ['m', 'f'], random_state=seed)


MemoryError: Unable to allocate 24.2 MiB for an array with shape (2, 1584636) and data type int64

In [36]:
balanced

,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,PWGTP,AGEP,...,MLPI,MLPJ,MLPK,VPS,ENG,LANP,FHINS5C,YOEP,DECADE,CITWP
28011,P,2018HU1396389,0,4,1101,9,72,1013097,98,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28012,P,2018HU1396389,0,5,1101,9,72,1013097,114,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28014,P,2018HU1396530,0,2,1102,9,72,1013097,186,19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28015,P,2018HU1396530,0,3,1102,9,72,1013097,186,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28017,P,2018HU1396899,0,2,804,9,72,1013097,72,48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28013,P,2018HU1396530,0,1,1102,9,72,1013097,123,39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28061,P,2018HU1398269,0,1,301,9,72,1013097,208,75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28069,P,2018HU1398656,0,2,101,9,72,1013097,102,73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28042,P,2018HU1397421,0,3,804,9,72,1013097,46,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
[balanced['AGEP']]

[28011    24
 28012     7
 28014    19
 28015    18
 28017    48
          ..
 28013    39
 28061    75
 28069    73
 28042    17
 28040    46
 Name: AGEP, Length: 92, dtype: int64]

In [53]:
import numbers

for i in partitioned_df_new['COW']:
     if isinstance(i, numbers.Number):
        print(i)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [45]:
acs_data['COW']

0        1.0
1        1.0
2        5.0
3        NaN
4        1.0
        ... 
28112    1.0
28113    NaN
28114    NaN
28115    NaN
28116    1.0
Name: COW, Length: 3236107, dtype: float64

In [37]:
balanced.shape

(92, 280)

In [ ]:
from sklearn.model_selection import train_test_split, KFold, ShuffleSplit
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.ensemble import RandomForestClassifier


metrics = {}
sample_p = np.linspace(0,1,11)
p_sel = 'SEX'
seed = 777
cv = 10
eval_sel = ['m', 'f']
model = 'randomforest'


################################################
#######            Thresholding         ########
################################################

# sample_t = np.linspace(0.45,0.55,11)
result_df = pd.DataFrame()
train_size = 4120

for idx, p in enumerate(sample_p):
    print('p={:.1f}'.format(p))
    ss = ShuffleSplit(n_splits=cv, test_size=0.3, random_state=seed)
    metric_cv = []

    fnr_list = [[], []]
    fpr_list = [[], []]
    acc_list = [[], []]
    i = 0
    
    for train, test in tqdm(ss.split(X)):
        X_train, X_test, y_train, y_test = X.iloc[train], X.iloc[test], y.iloc[train], y.iloc[test]
        np.random.seed(seed)
        X_train = select_rows_p(X_train, p_sel, p)
        print(X_train)
        y_train = y_train.loc[X_train.index]
        print(y_train)
#         pred_model = LogisticRegression(multi_class='ovr', C = 1)
#         pred_model.fit(X_train, y_train)
        pred_model = train_model(X_train, y_train, model)
        pred_model = RandomForestT(pred_model, t)
        
        metrics =  eval_binary_metrics(pred_model, X_test, y_test, eval_sel)
        metric_cv.append(metrics)
        for j in range(2):
            fnr_list[j].append(metrics[eval_sel[j]]['FNR'])
            fpr_list[j].append(metrics[eval_sel[j]]['FPR'])
            acc_list[j].append(metrics[eval_sel[j]]['Accuracy'])
            
        result_df = result_df.append({'t': t, 'iter': i, 'fnr0': fnr_list[0][-1], 'fpr0': fpr_list[0][-1], 
                                      'fnr1': fnr_list[1][-1], 'fpr1': fpr_list[1][-1], 
                                      'acc0': acc_list[0][-1], 'acc1': acc_list[1][-1] }, ignore_index=True)

        i+=1
 
       
################################################
#######  Training Set Composition       ########
################################################
X = df_new[x_labels]
y = df_new[pred_label]

sample_p = np.linspace(0,1,11)
cv = 30
result_df2 = pd.DataFrame()

for idx, p in enumerate(sample_p):
    print('p={:.1f}'.format(p))
    fnr_list = [[], []]
    fpr_list = [[], []]
    acc_list = [[], []]
    
    ss = ShuffleSplit(n_splits=cv, test_size=0.3, random_state=seed)
    metric_cv = []
    
    i = 0
    for train, test in tqdm(ss.split(X)):
        X_train, X_test, y_train, y_test = X.iloc[train], X.iloc[test], y.iloc[train], y.iloc[test]
        np.random.seed(seed)
        X_train = select_rows_p(X_train, p_sel, p)
        if i == 0:
            print(X_train.shape)
        y_train = y_train.loc[X_train.index]
        
        pred_model = train_model(X_train, y_train, model)

        metrics =  eval_binary_metrics(pred_model, X_test, y_test, eval_sel)
        metric_cv.append(metrics)

        for j in range(2):
            fnr_list[j].append(metrics[eval_sel[j]]['FNR'])
            fpr_list[j].append(metrics[eval_sel[j]]['FPR'])
            acc_list[j].append(metrics[eval_sel[j]]['Accuracy'])

        result_df2 = result_df2.append({'p': p, 'iter': i, 'fnr0': fnr_list[0][-1], 'fpr0': fpr_list[0][-1], 
                                      'fnr1': fnr_list[1][-1], 'fpr1': fpr_list[1][-1], 
                                      'acc0': acc_list[0][-1], 'acc1': acc_list[1][-1] }, ignore_index=True)

        i += 1
        
################################################
#######            Plotting             ########
################################################

f, axs = plt.subplots(2,3,figsize=(18,9))
colors = ['#081649', '#6C6C6C']
labels = ['F', 'M']
y_labels = ['fpr', 'fnr', 'acc']

for i in range(3):
    for j in range(2):
        y_label = y_labels[i]+str(j)
        sns.lineplot(data=result_df, x="t", y=y_label, ax = axs[0,i], color=colors[j], label= labels[j])
    axs[0,i].set_ylabel(y_labels[i])
    axs[0,i].set_ylim(0.1, 0.5)

axs[0,2].set_ylim(0.71, 0.77)

for i in range(3):
    for j in range(2):
        y_label = y_labels[i]+str(j)
        sns.lineplot(data=result_df2, x="p", y=y_label, ax = axs[1,i], color=colors[j], label= labels[j])
    axs[1,i].set_ylabel(y_labels[i])
    axs[1,i].set_ylim(0.1, 0.5)

axs[1,2].set_ylim(0.71, 0.77)

plt.savefig('thresh_vs_comp.pdf')
    

    
    

In [ ]:
group_train.shape

In [ ]:
group_test.shape

In [ ]:
group_train[2]

In [ ]:
group_test[1]

In [ ]:
X_train[0][2]

In [ ]:
y_train

In [ ]:
from sklearn.linear_model import LogisticRegression

model = make_pipeline(StandardScaler(), LogisticRegression())
model.fit(X_train, y_train)

yhat = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

male_tpr = np.mean(yhat[(y_test == 1) & (group_test == 1)])
female_tpr = np.mean(yhat[(y_test == 1) & (group_test == 2)])

male_tnr = np.mean(yhat[(y_test == 0) & (group_test == 1)] == 0)
female_tnr = np.mean(yhat[(y_test == 0) & (group_test == 2)] == 0)

male_accuracy = accuracy_score(y_test[group_test == 1], yhat[group_test == 1])
female_accuracy = accuracy_score(y_test[group_test == 2], yhat[group_test == 2])

In [ ]:
ACSIncomeNew.features

In [ ]:
sample_p = np.linspace(0,1,11)

pred_label = 'SEX'


# exclude_labels = ACSIncomeNew.features.copy()
# exclude_labels.remove(pred_label)

In [ ]:
eval_sel = ['Male', 'Female']
sample_t = np.linspace(0.45,0.55,11)
x_labels = ACSIncomeNew.features.copy()


In [ ]:
acs_data.describe()

In [ ]:
acs_data[acs_data['SEX']==2] # Female

In [ ]:
df_m_sub

In [ ]:
acs_data[acs_data['SEX']== 1] # Male

In [ ]:
acs_data[acs_data['SEX']== 2] # Female

In [ ]:
# def balance_data(df, groups, random_state=42):
    
#     df0 = select_rows(df, groups[0])
#     df1 = select_rows(df, groups[1])
#     size0, size1 = len(df0), len(df1)
#     if size0 > size1:
#         df_tmp = df0
#         df0 = df1
#         df1 = df_tmp
#     size0, size1 = len(df0), len(df1)
#     np.random.seed(random_state)
#     sample_idx = np.random.permutation(np.arange(size1))[:size0]
#     df_new = pd.concat([df0,  df1.iloc[sample_idx]])

#     return df_new

In [ ]:
X_train

In [ ]:
X_train.shape

In [ ]:
y_train.shape